In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
from scipy.stats import kendalltau,spearmanr
import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import networkx as nx
import time
import os.path as osp
from collections import defaultdict
import pandas as pd
# import xlsxwriter
from torch_geometric.nn import GCNConv,GATConv, SGConv, BatchNorm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.set_printoptions(precision=10)
#gra = "Email-Enron_gcn"
# gra="scale"
path = "data/"
loader = "result_eigen/"
dataset = "cit-DBLP"

In [ ]:
def actual_eigen(g):

    centrality_actual=nx.eigenvector_centrality_numpy(g)
    
    return centrality_actual

In [ ]:
def data_process(g, mode="test"):
    
    adj = list(g.edges())
    m = np.max(adj)
    
    features = torch.zeros(size=(m+1,1)).float()
    
    print("No.of nodes = ",m+1)
    print("No.of edges = ",len(adj))

    for u,_ in adj:
        features[u,0]+=1

    features = features/max(features)
    adj = torch.LongTensor(adj).t().contiguous()
 
    return adj.to(device), features.to(device)

In [ ]:
class Model(nn.Module):
    def __init__(self, nin, nhid1, nout, hid_l, out_l=1):
        super(Model, self).__init__()
        
        self.gc1 = GCNConv(in_channels= nin, out_channels= nhid1)
        self.bc1 = BatchNorm(nhid1)
        self.gc2 = GCNConv(in_channels= nhid1, out_channels= nout)
        self.bc2 = BatchNorm(nhid1)
        self.lay1 = nn.Linear(nout ,hid_l)
        self.l0 = nn.Linear(hid_l,hid_l)
        self.lb0 = nn.BatchNorm1d(hid_l)
        self.l1 = nn.Linear(hid_l,hid_l)
        self.lb1 = nn.BatchNorm1d(hid_l)
        self.lay2 = nn.Linear(hid_l ,out_l)
        self.active1 = nn.LeakyReLU(0.1)
        
        with torch.no_grad():
            self.gc1.weight = Parameter(nn.init.uniform_(torch.empty(nin,nhid1),a=0.0,b=1.0))
            self.gc1.bias = Parameter(nn.init.uniform_(torch.empty(nhid1),a=0.0,b=1.0))
            self.gc2.weight = Parameter(nn.init.uniform_(torch.empty(nhid1,nout),a=0.0,b=1.0))
            self.gc2.bias = Parameter(nn.init.uniform_(torch.empty(nout),a=0.0,b=1.0))
            self.lay1.weight = Parameter(nn.init.uniform_(torch.empty(hid_l, nout ),a=0.0,b=1.0))
            self.l0.weight = Parameter(nn.init.uniform_(torch.empty(hid_l, hid_l),a=0.0,b=1.0))
            self.l1.weight = Parameter(nn.init.uniform_(torch.empty(hid_l, hid_l),a=0.0,b=1.0))
            self.lay2.weight = Parameter(nn.init.uniform_(torch.empty(out_l,hid_l),a=0.0,b=1.0))


    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = self.bc1(x)
        x = self.gc2(x, adj)
        x = self.bc2(x)
        x = self.lay1(x)
        x = self.l0(x)
        x = self.lb0(x)
        x = self.l1(x)
        x = self.lb1(x)
        x = self.lay2(x)
        
        return self.active1(x)

In [ ]:
def test_model(features, adj):
    model.eval()
    with torch.no_grad():
        y = model(features, adj)
    return y

In [ ]:
def test(g):
    
    adj,features = data_process(g)
    net_time=time.time()

    y = test_model(features, adj)
    tn.append(time.time()-net_time)
    print("Time Net Test= ",time.time()-net_time)
    
    z = defaultdict(list)
    for i in range(len(y)):
        z[i] = y[i]
    x_net = list({k: v for k, v in sorted(z.items(), key=lambda item: item[1], reverse=True)}.keys())
    #print("x_net = ",x_net)
    
    #Actual Eigen
    print("$"*300)
    s1 = time.time()
    yy=actual_eigen(g)
    tr.append(time.time()-s1)

    x_actual = list({k: v for k, v in sorted(yy.items(), key=lambda item: item[1], reverse=True)}.keys())
    
    #print("\n\nx_actual = ",x_actual)
    per = [0.05, 0.1, 0.15, 0.2]
    
    for k in per:
    
        print("Top {}%:".format(k*100))  
        
        x_net_set = set(x_net[:int(k*len(features))])
        x_actual_set = set(x_actual[:int(k*len(features))])

        l = len(x_net_set.intersection(x_actual_set))
        print(l)

        ll = len(x_net_set)
        print(ll)

        acc = l/ll
        print(acc*100, "%\n\n")
        
        if k==0.05:
            l5.append(acc)
        elif k==0.1:
            l10.append(acc)
        elif k==0.15:
            l15.append(acc)
        elif k==0.2:
            l20.append(acc)
        print("%"*200)

model= Model(nin = 1, nhid1=128, nout=128, hid_l=64, out_l=1).to(device)
model.load_state_dict(torch.load(loader+dataset+".pt"))

In [ ]:
l5, l10, l15, l20 = [],[],[],[]
tn, tr = [],[]

g = nx.read_edgelist(osp.join(path,dataset+".edgelist"), nodetype=int)
test(g)
print("Average accuracy = ",np.mean(l5),np.mean(l10),np.mean(l15),np.mean(l20),sep="\n")